## Load Data

In [1]:
!pip install pyarrow
!pip install catboost

     |████████████████████████████████| 67.3MB 106kB/s 


In [2]:
import numpy as np
import pandas as pd 
import gc
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
train_raw = pd.read_parquet("./drive/MyDrive/Colab Notebooks/data/feature_u_m_um_b_c_mb_mc", engine="pyarrow")

In [ ]:
train_raw.head(10)

In [ ]:
train = train_raw.dropna()
del train_raw
gc.collect()

In [ ]:
train["gender"] = train["gender"].astype("int8")
train["age"] = train["age"].astype("int8")
train["m_popular_age"] = train["m_popular_age"].astype("int8")
train["m_popular_gender"] = train["m_popular_gender"].astype("int8")

## Feature Selection

### Nearly Zero Variance

In [ ]:
train_X, train_y = train.drop(["user_id", "merchant_id", "label", "item_id", "cat_id", "brand_id"], axis=1), train["label"]

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score

var_thres = VarianceThreshold(threshold=0.05)
var_thres.fit(train_X)

# 交叉验证
train_X_varthresh = train_X.iloc[:,var_thres.get_support(indices=True)]
print("使用方差阈值筛选出来的特征结果索引:",var_thres.get_support(indices=True))

使用方差阈值筛选出来的特征结果索引: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  29  30  31  32  33  34  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 103 104 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 182 183 184 185 186 187 188 189 190 191
 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 218 219 220 221 222 223 224 225 226 227 228 229 230
 231 232 235 239 240 242 243 244 245 246 247 248 250 251 252 254 255 256
 257 258 259 266 267 268 269 270 271 272 273 274 275 276 277 278 279 286
 290 291 292 293 294 295 302 303 310 318 320 322 323 324 325 326 327 340
 341 342 343 344 345 346 347 348

In [ ]:
train_X_varthresh.head(1)

,gender,age,u_ttl_clk_atc_cnt,u_ttl_buy_cnt,u_ttl_atf_cnt,u_clk_atc_cnt_m5,u_clk_atc_cnt_m6,u_clk_atc_cnt_m7,u_clk_atc_cnt_m8,u_clk_atc_cnt_m9,u_clk_atc_cnt_m10,u_clk_atc_cnt_m11,u_buy_cnt_m5,u_buy_cnt_m6,u_buy_cnt_m7,u_buy_cnt_m8,u_buy_cnt_m9,u_buy_cnt_m10,u_buy_cnt_m11,u_atf_cnt_m5,u_atf_cnt_m6,u_atf_cnt_m7,u_atf_cnt_m8,u_atf_cnt_m9,u_atf_cnt_m10,u_atf_cnt_m11,u_clk_atc_ratio_m5,u_clk_atc_ratio_m6,u_clk_atc_ratio_m7,u_clk_atc_ratio_m8,u_clk_atc_ratio_m9,u_clk_atc_ratio_m10,u_clk_atc_day_cnt,u_buy_day_cnt,u_atf_day_cnt,u_clk_atc_day_cnt_m5,u_clk_atc_day_cnt_m6,u_clk_atc_day_cnt_m7,u_clk_atc_day_cnt_m8,u_clk_atc_day_cnt_m9,u_clk_atc_day_cnt_m10,u_clk_atc_day_cnt_m11,u_buy_day_cnt_m5,u_buy_day_cnt_m6,u_buy_day_cnt_m7,u_buy_day_cnt_m8,u_buy_day_cnt_m9,u_buy_day_cnt_m10,u_buy_day_cnt_m11,u_atf_day_cnt_m5,u_atf_day_cnt_m6,u_atf_day_cnt_m7,u_atf_day_cnt_m8,u_atf_day_cnt_m9,u_atf_day_cnt_m10,u_atf_day_cnt_m11,u_clk_atc_item_div,u_buy_item_div,u_atf_item_div,u_clk_atc_cat_div,u_buy_cat_div,u_atf_cat_div,u_clk_atc_brd_div,u_buy_brd_div,u_atf_brd_div,u_clk_atc_mer_div,u_buy_mer_div,u_atf_mer_div,u_clk_atc_mean,u_buy_mean,u_atf_mean,u_clk_atc_median,u_buy_median,u_atf_median,u_clk_atc_std,u_buy_std,u_atf_std,u_clk_atc_max,u_buy_max,u_atf_max,u_clk_atc_day_cnt_mean,u_clk_atc_day_cnt_max,u_clk_atc_day_cnt_std,u_buy_day_cnt_max,u_buy_day_cnt_std,u_atf_day_cnt_mean,u_atf_day_cnt_max,u_atf_day_cnt_std,u_atf_day_cnt_med,u_clk_atc_item_cnt_mean,u_clk_atc_item_cnt_max,u_clk_atc_item_cnt_std,u_clk_atc_item_cnt_med,u_buy_item_cnt_mean,u_buy_item_cnt_max,u_buy_item_cnt_std,u_buy_item_cnt_med,u_atf_item_cnt_mean,u_atf_item_cnt_max,u_atf_item_cnt_std,u_atf_item_cnt_med,u_clk_atc_cnt_double11,u_buy_cnt_double11,u_atf_cnt_double11,u_clk_atc_cnt_1wpre_double11,u_buy_cnt_1wpre_double11,u_atf_cnt_1wpre_double11,u_clk_atc_ratio_double11,u_clk_atc_ratio_1wpre_double11,u_buy_ratio_double11,m_ttl_clk_atc_cnt,m_ttl_buy_cnt,m_ttl_atf_cnt,m_clk_atc_cnt_m5,m_clk_atc_cnt_m6,m_clk_atc_cnt_m7,m_clk_atc_cnt_m8,m_clk_atc_cnt_m9,m_clk_atc_cnt_m10,m_clk_atc_cnt_m11,m_buy_cnt_m5,m_buy_cnt_m6,m_buy_cnt_m7,m_buy_cnt_m8,m_buy_cnt_m9,m_buy_cnt_m10,m_buy_cnt_m11,m_atf_cnt_m5,m_atf_cnt_m6,m_atf_cnt_m7,m_atf_cnt_m8,m_atf_cnt_m9,m_atf_cnt_m10,m_atf_cnt_m11,m_clk_atc_day_cnt,m_buy_day_cnt,m_atf_day_cnt,m_clk_atc_day_cnt_m5,m_clk_atc_day_cnt_m6,m_clk_atc_day_cnt_m7,m_clk_atc_day_cnt_m8,m_clk_atc_day_cnt_m9,m_clk_atc_day_cnt_m10,m_clk_atc_day_cnt_m11,m_buy_day_cnt_m5,m_buy_day_cnt_m6,m_buy_day_cnt_m7,m_buy_day_cnt_m8,m_buy_day_cnt_m9,m_buy_day_cnt_m10,m_buy_day_cnt_m11,m_atf_day_cnt_m5,m_atf_day_cnt_m6,m_atf_day_cnt_m7,m_atf_day_cnt_m8,m_atf_day_cnt_m9,m_atf_day_cnt_m10,m_atf_day_cnt_m11,m_clk_atc_item_div,m_buy_item_div,m_atf_item_div,m_clk_atc_cat_div,m_buy_cat_div,m_atf_cat_div,m_clk_atc_brd_div,m_buy_brd_div,m_atf_brd_div,m_clk_atc_mean,m_buy_mean,m_atf_mean,m_clk_atc_median,m_buy_median,m_atf_median,m_clk_atc_std,m_buy_std,m_atf_std,m_clk_atc_max,m_buy_max,m_atf_max,m_clk_atc_day_cnt_mean,m_clk_atc_day_cnt_max,m_clk_atc_day_cnt_std,m_buy_day_cnt_max,m_atf_day_cnt_max,m_atf_day_cnt_std,m_clk_atc_item_cnt_mean,m_clk_atc_item_cnt_max,m_clk_atc_item_cnt_std,m_clk_atc_item_cnt_med,m_buy_item_cnt_mean,m_buy_item_cnt_max,m_buy_item_cnt_std,m_atf_item_cnt_mean,m_atf_item_cnt_max,m_atf_item_cnt_std,m_clk_atc_cnt_double11,m_buy_cnt_double11,m_atf_cnt_double11,m_clk_atc_cnt_1wpre_double11,m_buy_cnt_1wpre_double11,m_atf_cnt_1wpre_double11,m_rpt_byr_num,m_rpt_byr_day,m_popular_age,m_popular_gender,um_ttl_clk_atc_cnt,um_ttl_buy_cnt,um_ttl_atf_cnt,um_clk_atc_cnt_m5,um_clk_atc_cnt_m6,um_clk_atc_cnt_m7,um_clk_atc_cnt_m8,um_clk_atc_cnt_m9,um_clk_atc_cnt_m10,um_clk_atc_cnt_m11,um_buy_cnt_m11,um_atf_cnt_m8,um_atf_cnt_m9,um_atf_cnt_m10,um_atf_cnt_m11,um_ttl_clk_atc_ratio,um_ttl_buy_ratio,um_clk_atc_ratio_m10,um_clk_atc_ratio_m11,um_buy_ratio_m11,um_clk_atc_day_cnt,um_atf_day_cnt,um_clk_atc_day_cnt_m6,um_clk_atc_day_cnt_m7,um_clk_atc_day_cnt_m8,um_clk_atc_day_cnt_m9,um_clk_atc_day_cnt_m10,um_clk_a

### Correlation Analysis

In [ ]:
from scipy.stats import pearsonr

for i in range(train_X.shape[1]):
  feature1 = train_X.columns[i]
  for j in range(i+1, train_X.shape[1]):
    feature2 = train_X.columns[j]
    pearson_coefficient, p_value = pearsonr(train_X[feature1], train_X[feature2])
    if (pearson_coefficient >= 0.8 and p_value < 0.05):
      print("The correlation of {} and {} is {} and the p-value of the test is {}:".format(feature1, feature2, pearson_coefficient, p_value))

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


The correlation of u_ttl_clk_atc_cnt and u_clk_atc_item_div is 0.9555522692450107 and the p-value of the test is 0.0:
The correlation of u_ttl_clk_atc_cnt and u_clk_atc_brd_div is 0.8103496243225226 and the p-value of the test is 0.0:
The correlation of u_ttl_clk_atc_cnt and u_clk_atc_mer_div is 0.8070033931493095 and the p-value of the test is 0.0:
The correlation of u_ttl_clk_atc_cnt and u_clk_atc_mean is 0.9388097120007506 and the p-value of the test is 0.0:
The correlation of u_ttl_clk_atc_cnt and u_clk_atc_median is 0.8843437634451041 and the p-value of the test is 0.0:
The correlation of u_ttl_clk_atc_cnt and u_clk_atc_std is 0.8521048170081718 and the p-value of the test is 0.0:
The correlation of u_ttl_clk_atc_cnt and u_clk_atc_max is 0.8835001358968824 and the p-value of the test is 0.0:
The correlation of u_ttl_buy_cnt and u_buy_day_cnt is 0.86483964767171 and the p-value of the test is 0.0:
The correlation of u_ttl_buy_cnt and u_buy_item_div is 0.9767028056547542 and the p-v

### Low Importance 

In [ ]:
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.ensemble import RandomForestRegressor

X = train_X_varthresh
Y = train_y
names = train_X_varthresh.columns

#n_estimators为森林中树木数量，max_depth树的最大深度
rf = RandomForestRegressor(n_estimators=1000, max_depth=10)
scores = []
for i in range(X.shape[1]):
    #每次选择一个特征，进行交叉验证，训练集和测试集为8:2的比例进行分配，
    #ShuffleSplit()函数用于随机抽样（数据集总数，迭代次数，test所占比例）
    #3折交叉验证
    score = cross_val_score(rf, X.iloc[:, i:i+1], Y, scoring="roc_auc", cv=3)
    scores.append((round(np.mean(score), 3), names[i]))

打印出各个特征所对应的得分
print(sorted(scores, reverse=True))

[(0.631, 'm_clk_atc_cnt_m10'), (0.63, 'm_ttl_clk_atc_cnt'), (0.629, 'm_clk_atc_cnt_m7'), (0.629, 'm_clk_atc_cnt_m11'), (0.627, 'm_clk_atc_cnt_m5'), (0.625, 'm_clk_atc_cnt_m9'), (0.624, 'm_clk_atc_cnt_m8'), (0.623, 'm_ttl_atf_cnt'), (0.622, 'm_buy_cnt_m11'), (0.62, 'm_ttl_buy_cnt'), (0.62, 'm_clk_atc_cnt_m6'), (0.61, 'm_buy_cnt_m9'), (0.607, 'm_buy_cnt_m10'), (0.605, 'm_buy_cnt_m6'), (0.601, 'm_buy_cnt_m7'), (0.6, 'm_atf_cnt_m6'), (0.599, 'm_buy_cnt_m5'), (0.598, 'm_buy_cnt_m8'), (0.598, 'm_atf_cnt_m5'), (0.572, 'u_buy_item_cnt_mean'), (0.571, 'u_buy_item_cnt_max'), (0.565, 'u_clk_atc_item_cnt_mean'), (0.562, 'u_clk_atc_item_cnt_max'), (0.561, 'u_clk_atc_item_cnt_std'), (0.561, 'u_buy_item_cnt_std'), (0.558, 'u_ttl_buy_cnt'), (0.557, 'u_buy_item_div'), (0.556, 'u_clk_atc_day_cnt_max'), (0.554, 'u_clk_atc_day_cnt_std'), (0.554, 'u_clk_atc_day_cnt_mean'), (0.551, 'u_buy_day_cnt'), (0.55, 'u_buy_mean'), (0.549, 'u_buy_cat_div'), (0.548, 'u_clk_atc_mean'), (0.548, 'u_buy_max'), (0.546, 'u_c

## Soft-Voting

In [ ]:
train_X, train_y = train_X_varthresh.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x))), train['label']

In [ ]:
from tqdm import tqdm
import time
from sklearn.ensemble import VotingClassifier
clf0 = LogisticRegression(penalty='l2', solver='sag', C=10, max_iter=10000)

clf1 = xgb.XGBClassifier(
        max_depth=13, # raw8
        n_estimators=300,
        learning_rate= 0.05,
        min_child_weight=300, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        eta=0.3,    
        seed=42        
    )

clf2 = cat.CatBoostClassifier(learning_rate=0.02, 
                              iterations=5000, 
                              max_depth=8,
                              eval_metric='AUC', 
                              od_wait=50,
                              od_type='Iter', 
                              random_state=10, 
                              thread_count=8, 
                              l2_leaf_reg=1, 
                              verbose=False)

clf3 = lgb.LGBMClassifier(
        max_depth=12, # 8
        n_estimators=1000,
        min_child_weight=200, 
        colsample_bytree=0.8, 
        subsample=0.8, 
        learning_rate=0.05,    
        seed=42
    )

clf4 = RandomForestClassifier(n_estimators=80,
                              min_samples_split=120,
                              min_samples_leaf=40,
                              max_depth=13,
                              max_features="sqrt",
                              random_state=10)

# Soft-Voting
eclf = VotingClassifier(estimators=[('lr', clf0), ('xgb', clf1), ('cat', clf2), ('lgb', clf3), ('rf', clf4)], voting='soft', weights=[1,2,1,2,2])

# for clf, label in tqdm(zip([clf0, clf1, clf2, eclf], ['Logistics Regression', 'LightGBM', 'Random Forest', 'Soft-Voting'])):
scores = cross_val_score(eclf, train_X, train_y, cv=5, scoring='roc_auc')
print("AUC: %0.3f (+/- %0.5f)" % (scores.mean(), scores.std()))

AUC: 0.698 (+/- 0.00371)
